### Hypothesis Testing

Hypothesis Testing is a statistical method used to make inferences or decisions about a population based on sample data. It starts with a null hypothesis (H0), which represents a default stance or no effect, and an alternative hypothesis (H1 or Ha), which represents what we aim to prove or expect to find. The process involves using sample data to determine whether to reject the null hypothesis in favor of the alternative hypothesis, based on the likelihood of observing the sample data under the null hypothesis.

In this notebook we explore this using a website dataset. This is based on the performance of two themes (Dark or Light) on a website, our task is to find which theme performs better using Hypothesis Testing.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_data(path_to_csv_file):
    '''
    Reads csv files from specified paths

    Parameters:
            path to csv file locations
    Returns:
            dataframes of imported csv files
    '''
    # Read CSV files
    df = pd.read_csv(path_to_csv_file)
    
    return df

data = read_data('website_ab_test.csv')

In [4]:
def df_snapshot(df):
    '''
    Returns top 5 rows of the dataframe.

    Parameters:
        df (DataFrame): Pandas dataframe
    '''
    return display(df.head())

print(f'web data:')
df_snapshot(data)

web data:


,Theme,Click Through Rate,Conversion Rate,Bounce Rate,Scroll_Depth,Age,Location,Session_Duration,Purchases,Added_to_Cart
0,Light Theme,0.054920,0.282367,0.405085,72.489458,25,Chennai,1535,No,Yes
1,Light Theme,0.113932,0.032973,0.732759,61.858568,19,Pune,303,No,Yes
2,Dark Theme,0.323352,0.178763,0.296543,45.737376,47,Chennai,563,Yes,Yes
3,Light Theme,0.485836,0.325225,0.245001,76.305298,58,Pune,385,Yes,No
4,Light Theme,0.034783,0.196766,0.765100,48.927407,25,New Delhi,1437,No,No


In [5]:
def data_types(df):
    '''
    Returns datatype of fields in a dataframe.

    Parameters:
        df (DataFrame): Pandas dataframe
    '''
    return df.info()

print(f'web data:')
data_types(data)

web data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Theme               1000 non-null   object 
 1   Click Through Rate  1000 non-null   float64
 2   Conversion Rate     1000 non-null   float64
 3   Bounce Rate         1000 non-null   float64
 4   Scroll_Depth        1000 non-null   float64
 5   Age                 1000 non-null   int64  
 6   Location            1000 non-null   object 
 7   Session_Duration    1000 non-null   int64  
 8   Purchases           1000 non-null   object 
 9   Added_to_Cart       1000 non-null   object 
dtypes: float64(4), int64(2), object(4)
memory usage: 78.3+ KB


In [7]:
def null_checker(df):
    '''
    Returns the count of rows found in the data frame where there is a null present.

    Parameters:
        df (Dataframe): Pandas dataframe

    Returns:
        Prints the null count, null percentage, and row count per column.
    '''
    row_count = len(df)
    null_counts = df.isnull().sum().sort_values(ascending=False)
    null_percentages = (null_counts / row_count) * 100
    combined = pd.concat([null_counts, null_percentages], axis=1, keys=['Null Count', 'Null Percentage'])
    combined['Row Count'] = row_count
    print(combined)

# check nulls in train
print(f'web data:')
null_checker(data)

web data:
                    Null Count  Null Percentage  Row Count
Theme                        0              0.0       1000
Click Through Rate           0              0.0       1000
Conversion Rate              0              0.0       1000
Bounce Rate                  0              0.0       1000
Scroll_Depth                 0              0.0       1000
Age                          0              0.0       1000
Location                     0              0.0       1000
Session_Duration             0              0.0       1000
Purchases                    0              0.0       1000
Added_to_Cart                0              0.0       1000


In [8]:
data.describe()

,Click Through Rate,Conversion Rate,Bounce Rate,Scroll_Depth,Age,Session_Duration
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.256048,0.253312,0.505758,50.319494,41.528000,924.999000
std,0.139265,0.139092,0.172195,16.895269,14.114334,508.231723
min,0.010767,0.010881,0.200720,20.011738,18.000000,38.000000
25%,0.140794,0.131564,0.353609,35.655167,29.000000,466.500000
50%,0.253715,0.252823,0.514049,51.130712,42.000000,931.000000
75%,0.370674,0.373040,0.648557,64.666258,54.000000,1375.250000
max,0.499989,0.498916,0.799658,79.997108,65.000000,1797.000000


In [12]:
def analyze_theme_performance(df, theme_column='Theme', metric_column='Conversion Rate'):
    """
    Groups data by theme, calculates mean values for the metrics,
    sorts by the specified metric, and prints the sorted DataFrame.

    """
    theme_performance = df.groupby(theme_column).mean()
    theme_performance_sorted = theme_performance.sort_values(by=metric_column, ascending=False)

    return theme_performance_sorted

analyze_theme_performance(data)

,Click Through Rate,Conversion Rate,Bounce Rate,Scroll_Depth,Age,Session_Duration
Theme,,,,,,
Light Theme,0.247109,0.255459,0.499035,50.735232,41.734568,930.833333
Dark Theme,0.264501,0.251282,0.512115,49.926404,41.332685,919.482490


#### The comparison between the Light Theme and Dark Theme on average performance metrics reveals the following insights:

- Click Through Rate (CTR): The Dark Theme has a slightly higher average CTR (0.2645) compared to the Light Theme (0.2471).
- Conversion Rate: The Light Theme leads with a marginally higher average Conversion Rate (0.2555) compared to the Dark Theme (0.2513).
- Bounce Rate: The Bounce Rate is slightly higher for the Dark Theme (0.5121) than for the Light Theme (0.4990).
- Scroll Depth: Users on the Light Theme scroll slightly further on average (50.74%) compared to those on the Dark Theme (49.93%).
- Age: The average age of users is similar across themes, with the Light Theme at approximately 41.73 years and the Dark Theme at 41.33 years.
- Session Duration: The average session duration is slightly longer for users on the Light Theme (930.83 seconds) than for those on the Dark Theme (919.48 seconds).

From these insights, it appears that the Light Theme slightly outperforms the Dark Theme in terms of Conversion Rate, Bounce Rate, Scroll Depth, and Session Duration, while the Dark Theme leads in Click Through Rate. However, the differences are relatively minor across all metrics.

#### Conversion rate hypothesis testing
We’ll use a significance level (alpha) of 0.05 for the hypothesis testing. It means we’ll consider a result statistically significant if the p-value from our test is less than 0.05. 

- Null Hypothesis (H0​): There is no difference in Conversion Rates between the Light Theme and Dark Theme.
- Alternative Hypothesis (Ha​): There is a difference in Conversion Rates between the Light Theme and Dark Theme.

We’ll use a two-sample t-test to compare the means of the two independent samples. 

- If the p-value ≤ 0.05: There is strong evidence against the null hypothesis, so we reject the null hypothesis. This means the results are statistically significant, and there is a difference in Conversion Rates between the Light Theme and Dark Theme.
- If the p-value > 0.05: There is not enough evidence to reject the null hypothesis. This means the results are not statistically significant, and there is no difference in Conversion Rates between the Light Theme and Dark Theme.

In [26]:
def compare_themes(data, theme_column='Theme', metrics=['Click Through Rate', 'Conversion Rate', 'Bounce Rate', 'Scroll_Depth'], themes=['Light Theme', 'Dark Theme']):
    """
    Compares multiple metrics between two themes using two-sample t-tests and creates a comparison table.
    """
    results = {'Metric': [], 'T-Statistic': [], 'P-Value': []}
    
    for metric in metrics:
        data_theme1 = data[data[theme_column] == themes[0]][metric]
        data_theme2 = data[data[theme_column] == themes[1]][metric]
        
        t_stat, p_value = ttest_ind(data_theme1, data_theme2, equal_var=False)
        
        results['Metric'].append(metric)
        results['T-Statistic'].append(t_stat)
        results['P-Value'].append(p_value)
    
    comparison_table = pd.DataFrame(results)
    
    return comparison_table

# Example usage:
comparison_table = compare_themes(data)
comparison_table

,Metric,T-Statistic,P-Value
0,Click Through Rate,-1.978171,0.048184
1,Conversion Rate,0.474849,0.634998
2,Bounce Rate,-1.201888,0.229692
3,Scroll_Depth,0.756228,0.449692


- Conversion Rate: No statistical significance (P-value=0.634). Since this p-value is much greater than our significance level of 0.05, we do not have enough evidence to reject the null hypothesis. Therefore, we conclude that there is no statistically significant difference in Conversion Rates between the Light Theme and Dark Theme based on the data provided.
- Click-Through Rate: Statistically significant (P-value=0.048). This p-value is slightly below the significance level of 0.05, indicating that there is a statistically significant difference in Click Through Rates between the Light Theme and Dark Theme, with the Dark Theme likely having a higher CTR given the direction of the test statistic.
- Bounce Rate: There’s no statistically significant difference in Bounce Rates between the themes (P-Value = 0.230).
- Scroll Depth: Statistically significant difference is observed in Scroll Depths (P-Value = 0.450), indicating that there is a statistically significant difference in how much further users scroll on the website between the Light Theme and Dark Theme.


In summary, while the two themes perform similarly across most metrics, the Dark Theme has a slight edge in terms of engaging users to click through. For other key performance indicators like Conversion Rate, Bounce Rate, and Scroll Depth, the choice between a Light Theme and a Dark Theme does not significantly affect user behaviour according to the data provided.